# Reflexion Agent

## Workflow of a Reflexion Agent in LangGraph
1. Generation Node: Produce Initial Attempt
The agent generates a first response based on the prompt, prioritizing speed and creativity over perfection — for example, drafting a quick YouTube video script idea to work with.

2. Reflection Node: Self-Critique and Plan
The agent reviews its own output, identifying weaknesses, gaps, or missed opportunities. It creates an improvement plan — for instance, noting that the video script needs a stronger hook or clearer structure.

3. Revision Node: Implement Improvements
Using its self-critique, the agent revises the output. This may involve restructuring the content, adding missing details, or improving tone and flow until it aligns with the set goals.

4. Evaluation Node: Check Against Goals
The improved draft is assessed against predefined criteria (e.g., engagement, coherence, audience fit). If it passes, the workflow ends; if not, it loops back for further reflection and revision.

5. Final Output: Deliver Enhanced Result
Once the content meets the desired standard, the agent outputs a polished version — for example, a refined YouTube video script ready for production or review.



![Refexion Agent](../.github/assets/reflexion_agent.png)

In [1]:
import json
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from langgraph.graph import END, MessageGraph
from langchain.tools import Tool
from ddgs import DDGS

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate

summary_llm = init_chat_model(model='claude-3-haiku-20240307', model_provider='anthropic', temperature=0.0)
llm = init_chat_model(model='claude-3-5-haiku-latest', model_provider='anthropic', temperature=0.0)

summary_prompt = PromptTemplate.from_template(
    "Summarize the content in 4-6 sentences relevant to the query. "
    "Return only your summary and nothing else. If their is no relevant content or the content is blocked, return 'no content'.\n\n"
    "Query:{query}\n\n"
    "Content:\n\n"
    "{content}"
)

summarizer = summary_prompt | summary_llm

In [3]:
import requests
from bs4 import BeautifulSoup

def fetch_and_clean(url):
    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        return soup.get_text()
    except Exception:
        return ""


In [4]:
def duckduckgo_extended(query: str, max_results=3) -> list:
    results = {'search':[]}
    fetched_pages = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, region="us-en", max_results=max_results):
            r.pop("body")
            fetched_pages.append(fetch_and_clean(r['href']))
            results['search'].append(r)
    summary = summarizer.invoke({'query': query, 'content': "\n\n\n".join(fetched_pages)})
    results['summary'] = {'summary': summary.content}
    return results
    
    

DuckDuckGoExtendedTool = Tool(
    name="DuckDuckGo Extended Search",
    func=duckduckgo_extended,
    description="Searches DuckDuckGo and retrieves up to N results."
)

In [5]:
result = DuckDuckGoExtendedTool.invoke("AI regulation in Japan")
result

{'search': [{'title': 'Understanding AI Regulations in Japan - Current Status and ...',
   'href': 'https://www.dlapiper.com/en/insights/publications/2024/10/understanding-ai-regulations-in-japan-current-status-and-future-prospects'},
  {'title': 'AI Watch: Global regulatory tracker - Japan | White & Case LLP',
   'href': 'https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-japan'},
  {'title': 'Understanding Japan\'s AI Promotion Act: An "Innovation-First ...',
   'href': 'https://fpf.org/blog/understanding-japans-ai-promotion-act-an-innovation-first-blueprint-for-ai-regulation/'}],
 'summary': {'summary': 'Here is a 4-6 sentence summary of the key points regarding AI regulation in Japan:\n\nJapan has enacted the "Act on Promotion of Research and Development and Utilization of Artificial Intelligence-Related Technologies", its first comprehensive AI legislation. The Act establishes core principles for AI development and use, creates an AI Strategy Center,

In [6]:
question="AI regulation in Japan"

In [7]:
prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a highly capable research assistant. Your role is to help users explore questions and problems by reasoning step-by-step, providing well-supported answers, and suggesting relevant web search queries to gather additional context.

        Your response must follow these steps:
        1. {first_instruction}
        2. Provide a thorough, reasoned answer to the user’s question using clear logic and, where applicable, established knowledge.
        3. Identify areas where additional, up-to-date (today"s date: {date_today}) information could improve the answer.
        4. Reflect on and critique your reasoning, noting any limitations or uncertainties.
        5. After the reflection, **list 1–3 search queries separately** that could help find the most relevant and recent information on this topic. Do not include them inside the reflection.

        Keep the tone professional, concise, and evidence-oriented.

        Answer the user's question below using the required structured format."
        """
    ),
    MessagesPlaceholder(variable_name="messages")
])

In [8]:
from datetime import datetime

In [9]:
datetime.now().strftime("%Y-%m-%d")

'2025-07-31'

In [10]:
first_responder_prompt = prompt_template.partial(first_instruction="Provide a detailed ~250 word answer", date_today=datetime.now().strftime("%Y-%m-%d"))
temp_chain = first_responder_prompt| llm
response = temp_chain.invoke({"messages": [HumanMessage(content=question)]})
print(response.content)

Japan has been taking a proactive and nuanced approach to AI regulation, positioning itself as a leader in developing comprehensive yet balanced governance frameworks for artificial intelligence. The country's strategy focuses on balancing technological innovation with ethical considerations and societal protection.

In 2019, Japan was among the first nations to develop national AI principles, emphasizing human-centric AI that respects fundamental human rights and promotes social trust. The government's approach prioritizes three key areas: promoting AI technological development, ensuring safety and reliability, and addressing potential socioeconomic disruptions.

The Japanese government has established multiple regulatory bodies, including the Strategic Council for AI Technology and the AI Network Society Promotion Council, which collaborate to draft guidelines and policy recommendations. These bodies focus on creating frameworks that encourage responsible AI development while maintai

In [11]:
class Reflection(BaseModel):
	missing: str = Field(description="What information is missing")
	superfluous: str = Field(description="What information is unnecessary")

class AnswerQuestion(BaseModel):
	answer: str = Field(description="Main response to the question")
	reflection: Reflection = Field(description="Self-critique of the answer")
	search_queries: List[str] = Field(description="Queries for additional research")

In [12]:
initial_chain = first_responder_prompt| llm.bind_tools(tools=[AnswerQuestion])
response=initial_chain.invoke({"messages":[HumanMessage(question)]})
print("---Full Structured Output---")
print(response.tool_calls)

---Full Structured Output---
[{'name': 'AnswerQuestion', 'args': {'answer': "Japan has been taking a proactive and nuanced approach to AI regulation, positioning itself as a global leader in responsible AI development. The Japanese government has developed a comprehensive strategy that balances innovation with ethical considerations and societal protection.\n\nIn 2023, Japan's approach to AI regulation has been characterized by collaborative governance, involving multiple stakeholders including government agencies, tech companies, and academic institutions. The primary framework for AI regulation centers on principles of human-centric AI that prioritizes human rights, transparency, and accountability.\n\nKey aspects of Japan's AI regulatory approach include:\n\n1. Ethical Guidelines: The government has established detailed ethical guidelines for AI development, emphasizing principles of fairness, transparency, and respect for human dignity. These guidelines are designed to be flexible 

In [13]:
answer_content = response.tool_calls[0]['args']['answer']
print("---Initial Answer---")
print(answer_content)

---Initial Answer---
Japan has been taking a proactive and nuanced approach to AI regulation, positioning itself as a global leader in responsible AI development. The Japanese government has developed a comprehensive strategy that balances innovation with ethical considerations and societal protection.

In 2023, Japan's approach to AI regulation has been characterized by collaborative governance, involving multiple stakeholders including government agencies, tech companies, and academic institutions. The primary framework for AI regulation centers on principles of human-centric AI that prioritizes human rights, transparency, and accountability.

Key aspects of Japan's AI regulatory approach include:

1. Ethical Guidelines: The government has established detailed ethical guidelines for AI development, emphasizing principles of fairness, transparency, and respect for human dignity. These guidelines are designed to be flexible enough to accommodate technological innovation while providing

In [14]:
Reflection_content = response.tool_calls[0]['args']['reflection']
print("---Reflection Answer---")
print(Reflection_content)

---Reflection Answer---
{'missing': 'Specific details about recent legislative actions, exact dates of key regulatory decisions, and precise mechanisms of enforcement', 'superfluous': "Overly technical details about specific AI technologies that don't directly relate to regulation"}


In [15]:
search_queries = response.tool_calls[0]['args']['search_queries']
print("---Search Queries---")
print(search_queries)

---Search Queries---
['Japan AI regulation 2024 policy', 'Japanese government AI ethical guidelines', 'METI AI regulation framework']


In [16]:
response_list=[]
response_list.append(HumanMessage(content=question))
response_list.append(response)

In [17]:
tool_call=response.tool_calls[0]
search_queries = tool_call["args"].get("search_queries", [])
print(search_queries)

['Japan AI regulation 2024 policy', 'Japanese government AI ethical guidelines', 'METI AI regulation framework']


In [18]:
search_tool = Tool(
    name="DuckDuckGo Extended Search",
    func=duckduckgo_extended,
    description="Searches DuckDuckGo and retrieves up to N results."
)

def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    last_ai_message = state[-1]
    tool_messages = []
    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]:
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("search_queries", [])
            query_results = {}
            for query in search_queries:
                result = search_tool.invoke(query)
                query_results[query] = result
            tool_messages.append(ToolMessage(
                content=json.dumps(query_results),
                tool_call_id=call_id)
            )
    return tool_messages

In [19]:
tool_response = execute_tools(response_list)
# Use .extend() to add all tool messages from the list
response_list.extend(tool_response)

In [20]:
tool_response

[ToolMessage(content='{"Japan AI regulation 2024 policy": {"search": [{"title": "Center for Strategic and International Studies New Government Policy Shows Japan Favors a Light Touch for AI Regulation | CSIS", "href": "https://www.csis.org/analysis/new-government-policy-shows-japan-favors-light-touch-ai-regulation"}, {"title": "White & Case LLP AI Watch: Global regulatory tracker - Japan | White & Case LLP", "href": "https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-japan"}, {"title": "Morrison & Foerster Japan\\u2019s Approach to AI Regulation in 2025 | MoFo Tech", "href": "https://mofotech.mofo.com/topics/japan-s-approach-to-ai-regulation-in-2025"}], "summary": {"summary": "Here is a 4-6 sentence summary of the relevant content:\\n\\nJapan has adopted a cautious, sector-specific approach to AI regulation, favoring voluntary industry initiatives over sweeping new laws. The government has established an AI Strategy Center to monitor emerging risks and pr

In [21]:
response_list

[HumanMessage(content='AI regulation in Japan', additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'id': 'toolu_01MBpZpAPEPHhEMCXLB4Zk9C', 'input': {'answer': "Japan has been taking a proactive and nuanced approach to AI regulation, positioning itself as a global leader in responsible AI development. The Japanese government has developed a comprehensive strategy that balances innovation with ethical considerations and societal protection.\n\nIn 2023, Japan's approach to AI regulation has been characterized by collaborative governance, involving multiple stakeholders including government agencies, tech companies, and academic institutions. The primary framework for AI regulation centers on principles of human-centric AI that prioritizes human rights, transparency, and accountability.\n\nKey aspects of Japan's AI regulatory approach include:\n\n1. Ethical Guidelines: The government has established detailed ethical guidelines for AI development, emphasizing principles of fa

In [22]:
revise_instructions = """
Revise your previous answer using the new information, applying a rigorous, evidence-based approach.
- Incorporate the previous critique to add relevant, up-to-date information, focusing on a clear mechanistic or logical understanding of the topic.
- You MUST include numerical citations referencing credible sources such as peer-reviewed research, official reports, or authoritative databases.
- Clearly distinguish between correlation and causation, and acknowledge uncertainties or limitations in the available information.
- When relevant, consider quantitative or measurable factors (e.g., key metrics, indicators, or benchmarks).
- Add a "References" section at the bottom of your answer in the form of:
  [1] https://example.com
  [2] https://example.com
- Use the critique to refine the answer, removing unsupported speculation and ensuring all claims are backed by high-quality evidence.
- Keep the response under 250 words, prioritizing precision and clarity over volume.
"""
revisor_prompt = prompt_template.partial(first_instruction=revise_instructions, date_today=datetime.now().strftime("%Y-%m-%d"))

In [23]:
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question."""
    references: List[str] = Field(description="Citations motivating your updated answer.")
revisor_chain = revisor_prompt | llm.bind_tools(tools=[ReviseAnswer])

In [24]:
response = revisor_chain.invoke({"messages": response_list})
print(response)
print("---Revised Answer with References---")
print(response.tool_calls[0]['args'])

content=[{'id': 'toolu_01E5fHA59Xcq9EfZPEkdxMXE', 'input': {'answer': 'Japan\'s AI regulation strategy in 2024-2025 represents a nuanced, innovation-friendly approach that balances technological advancement with ethical considerations. The government has adopted a "light-touch" regulatory framework characterized by:\n\n1. Soft Law Approach: Instead of rigid legislation, Japan favors voluntary industry guidelines and administrative guidance. The Ministry of Economy, Trade and Industry (METI) has developed comprehensive AI guidelines that promote responsible innovation without stifling technological progress [1].\n\n2. Sector-Specific Frameworks: Regulations are tailored to specific domains like healthcare, finance, and autonomous technologies, allowing for flexible, context-aware governance [2].\n\n3. International Collaboration: Japan is actively participating in global AI governance discussions, working to develop harmonized standards through platforms like the OECD [3].\n\nKey regula

In [25]:
response_list.append(response)

In [26]:
MAX_ITERATIONS = 2

In [27]:
def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations >= MAX_ITERATIONS:
        return END
    return "execute_tools"

In [28]:
graph=MessageGraph()


graph.add_node("respond", initial_chain)
graph.add_node("execute_tools", execute_tools)
graph.add_node("revisor", revisor_chain)

graph.add_edge("respond", "execute_tools")
graph.add_edge("execute_tools", "revisor")

graph.add_conditional_edges("revisor", event_loop)
graph.set_entry_point("respond")

app = graph.compile()

In [29]:
question

'AI regulation in Japan'

In [30]:
responses = app.invoke(
    question, 
    {"recursion_limit": 100}
)

In [31]:
print("--- Initial Draft Answer ---")
initial_answer = responses[1].tool_calls[0]['args']['answer']
print(initial_answer)
print("\n")

print("--- Intermediate and Final Revised Answers ---")
answers = []

# Loop through all messages in reverse to find all tool_calls with answers
for msg in reversed(responses):
    if getattr(msg, 'tool_calls', None):
        for tool_call in msg.tool_calls:
            answer = tool_call.get('args', {}).get('answer')
            if answer:
                answers.append(answer)

# Print all collected answers
for i, ans in enumerate(answers):
    label = "Final Revised Answer" if i == 0 else f"Intermediate Step {len(answers) - i}"
    print(f"{label}:\n{ans}\n")

--- Initial Draft Answer ---
Japan has been taking a proactive and nuanced approach to AI regulation, positioning itself as a global leader in responsible AI development. The Japanese government has developed a comprehensive strategy that balances innovation with ethical considerations and societal protection.

In 2019, Japan was one of the first countries to develop national AI principles, emphasizing human-centric AI that respects human rights and promotes social well-being. The government's AI strategy focuses on three key pillars: promoting AI research and development, ensuring AI's social implementation, and addressing ethical and legal challenges.

The Japanese approach differs from more restrictive models like the EU's AI Act. Instead of heavy-handed regulation, Japan advocates for a collaborative framework that encourages responsible innovation. The government has established expert panels and working groups that bring together academics, industry leaders, and policymakers to d